In [2]:
import numpy as np #creating numpy arrays
import pandas as pd #creating dataframes and storing data in dataframes
import re #for searching for text in a document
from nltk.corpus import stopwords  #removes words that don't add value to the article
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer #converts texts into numbers
from sklearn.model_selection import train_test_split #splits data into training and testing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\migue\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [81]:
news_dataset_train = pd.read_csv('/Users/migue/Desktop/TFM_Git/Dossier/modelo/datasets/Espanhol/train.csv')
news_dataset_test = pd.read_csv('/Users/migue/Desktop/TFM_Git/Dossier/modelo/datasets/Espanhol/test.csv')
news_dataset_test.shape
news_dataset_train.shape

news_dataset_train.head()

,Id,label,Topic,Source,Headline,Text,Link
0,1,Fake,Education,El Ruinaversal,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",http://www.elruinaversal.com/2017/06/10/rae-in...
1,2,Fake,Education,Hay noticia,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea...",https://haynoticia.es/la-palabra-haiga-aceptad...
2,3,Fake,Education,El Ruinaversal,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,http://www.elruinaversal.com/2018/05/06/yordi-...
3,4,True,Education,EL UNIVERSAL,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...,http://www.eluniversal.com.mx/articulo/nacion/...
4,5,Fake,Education,Lamula,pretenden aprobar libros escolares con conteni...,Alerta: pretenden aprobar libros escolares con...,https://redaccion.lamula.pe/2018/06/19/memoria...


In [82]:
news_dataset_train.drop('Link', axis=1, inplace=True)
news_dataset_train.drop('Source', axis=1, inplace=True)
news_dataset_train.drop('Topic', axis=1, inplace=True)

news_dataset_train.head()

,Id,label,Headline,Text
0,1,Fake,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR..."
1,2,Fake,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea..."
2,3,Fake,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...
3,4,True,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...
4,5,Fake,pretenden aprobar libros escolares con conteni...,Alerta: pretenden aprobar libros escolares con...


In [30]:
news_dataset.isnull().sum()

Id          0
label       0
Topic       0
Source      0
Headline    0
Text        0
Link        0
dtype: int64

In [83]:
news_dataset_train_label = news_dataset_train.replace({'True': 1, 'Fake': 0})
news_dataset_train_label.head()

,Id,label,Headline,Text
0,1,0,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR..."
1,2,0,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea..."
2,3,0,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...
3,4,1,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...
4,5,0,pretenden aprobar libros escolares con conteni...,Alerta: pretenden aprobar libros escolares con...


In [84]:
X = news_dataset_train_label.drop(columns='label', axis=1)
Y = news_dataset_train_label['label']
print(X)
print(Y)

      Id                                           Headline  \
0      1  RAE INCLUIRÁ LA PALABRA "LADY" EN EL DICCIONAR...   
1      2            La palabra "haiga", aceptada por la RAE   
2      3  YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...   
3      4  UNAM capacitará a maestros para aprobar prueba...   
4      5  pretenden aprobar libros escolares con conteni...   
..   ...                                                ...   
671  672  Madonna será la nueva imagen del Chocolate Abu...   
672  673  Filtran material íntimo de Zelina Vega, luchad...   
673  674  CBS PLANEA GRAN HERMANO EN LA CASA BLANCA CON ...   
674  675  TV Azteca anuncia que cambiará su nombre por u...   
675  676          MUERTE DE PETER EL ANGUILA POR SOBREDOSIS   

                                                  Text  
0    RAE INCLUIRÁ LA PALABRA "LADY" EN EL DICCIONAR...  
1    La palabra "haiga", aceptada por la RAE La Rea...  
2    YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...  
3    UNAM capac